# FDTD testing notebook

### Description

This notebook implements simple tests of the FDTD solver class written in Python.


Following tests are implemented:

1. Eigenmodes of a parallelepiped cavity with perfect cundoctor as a boundary.
2. Visual inspection of an initialization of the electric field to that of a point charge.
3. Visual inspection of an initialization of the electric field to that of a point charge *but* with the origin of the grid shifted by a given amount.

## Tests

Necessary imports of general libraries. We import NumPy, SciPy, Matplotlib and holoviews and set Matplotlib plots to be interactive widgets.

In [ ]:
import numpy as np
import scipy as sp

#%matplotlib notebook
%matplotlib inline
#%load_ext autoreload
#%autoreload 2
from matplotlib import pyplot as plt

import holoviews as hv
from scipy import constants

Import the grid class. This class does not diffeentiate between the field components in the array allocation and size.

In [ ]:
from fdtd3d import UnifiedYeeGrid

Define physical quantities in SI units from `scipy.constants`

In [ ]:
eps0 = constants.epsilon_0 
mu0 = constants.mu_0 
c0= constants.c 

We define a cavity of dimensions $L_x \times L_y \times L_z$ and discretize it using $N_x \times N_y \times N_z$ cells.

The fields will be initialized to white noise and propagated for $N_t$ time steps. Using the resulting sampled time-series of the electric field we can compute the empirical eigenfrequencies of the cavity, to be compared with TE and TM modes which can be determined analytically.

In [ ]:
Lx = .05;
Ly = .04;
Lz = .03;
Nx = 25;
Ny = 20;
Nz = 15;
Cx = Nx / Lx;
Cy = Ny / Ly;
Cz = Nz / Lz;
Nt = 8192;
Dt = 1/(c0*np.linalg.norm([Cx, Cy, Cz]))

In [ ]:
grid = UnifiedYeeGrid(Nx, Ny, Nz, pmlCells=(0,0,0), cellDim=(1./Cx, 1./Cy, 1./Cz) )

Initialize the electric field to white noise.

In [ ]:
grid.initRandom()

Create an array to store sampled field values to , to be later Fourier transformed into a spectrum.

In [ ]:
Et = np.zeros((Nt,3));

In [ ]:
for t in range(Nt):
            grid.updateOwn(eps0, mu0, Dt, (1./Cx, 1./Cy, 1./Cz) );
            Et[t, :] = np.array([ grid.Ex(3,3,3), grid.Ey(3,3,3), grid.Ez(3,3,3) ])

In [ ]:
t = np.linspace(0, Nt*Dt, Nt)

In [ ]:
ftFields = np.fft.fft(Et, axis=0)

In [ ]:
ftFields.shape

In [ ]:
frequencies=np.fft.fftfreq(Et.shape[0], d=Dt)

In [ ]:
angularFrequencies=2*np.pi*frequencies

Compute the analyti eigenfrequencies of the TE and TM modes of the cavity.

In [ ]:
def eigenfrequency(m, n, p, Lx, Ly, Lz):
    return c0/2 * np.sqrt( (m/Lx)**2 + (n/Ly)**2 + (p/Lz)**2 )

In [ ]:
eigenfreqList = []

xModes = 4
yModes = 3
zModes = 3

# TM modes
for m in range(1, xModes):
    for n in range(1, yModes):
        for p in range(zModes):
            eigenfreqList.append( eigenfrequency(m,n,p, Lx, Ly, Lz) )
# TE modes
for p in range(1, zModes):
    for n in range(yModes):
        if n==0:
            for m in range(1, xModes):
                eigenfreqList.append( eigenfrequency(m,n,p, Lx, Ly, Lz) )
        else:
            eigenfreqList.append( eigenfrequency(0,n,p, Lx, Ly, Lz) )

In [ ]:
plt.close()
plt.plot(np.fft.fftshift(frequencies/1e9), np.fft.fftshift(abs(np.sum(ftFields, axis=1))), 'r-')

plt.vlines([item/1e9 for item in np.unique(eigenfreqList)], 0, 50, linestyles='-.')
plt.xlim(0,10)
plt.ylim(0,50)

In [ ]:
plt.close()

In [ ]:
subset = abs(np.sum(ftFields, axis=1))[:len(ftFields)//2]

In [ ]:
subsetFrequencies = frequencies[:len(ftFields)//2]/1e9

Find the peaks in the signal, to filter the array of computed data to be compared to analytical frequencies.

In [ ]:
import scipy.signal as spsig

In [ ]:
peaks, _ = spsig.find_peaks(abs(np.sum(ftFields, axis=1))[:len(ftFields)//2], height=3)

In [ ]:
plt.close()
plt.plot(np.fft.fftshift(frequencies/1e9), np.fft.fftshift(abs(np.sum(ftFields, axis=1))), 'r-', label='$\propto|E_x + E_y+E_z|$')
#plt.plot(np.fft.fftshift(frequencies/1e9), abs(np.sum(ftFields, axis=1)), 'r-',)

plt.plot(subsetFrequencies[peaks], subset[peaks], "x", label='num. peakfinding')

plt.vlines([item/1e9 for item in np.unique(eigenfreqList)], 0, 50, linestyles='-.', label='analytic')

plt.xlabel("Frequency [GHz]", fontsize='x-large')
plt.ylabel("Field magnitude [V/m]", fontsize='x-large')
plt.legend(loc='upper left')
plt.xlim(1,10)
plt.ylim(0,50)
plt.savefig("PEC_Cavity_Eigenfrequencies.png", dpi=120)

In [ ]:
np.unique(subsetFrequencies[peaks])[:len(np.unique(eigenfreqList))]

In [ ]:
np.unique(eigenfreqList)/1e9

In [ ]:
EigenfreqDifferences = np.unique(subsetFrequencies[peaks])[:len(np.unique(eigenfreqList))] - np.unique(eigenfreqList)/1e9

In [ ]:
relApproxErr = EigenfreqDifferences/(np.unique(eigenfreqList)/1e9)

In [ ]:
print("Maximum relative error", abs(relApproxErr).max())
print("Minimum relative error", abs(relApproxErr).min())

## Plot field slices using Holoviews

In [ ]:
hv.extension('matplotlib')
#hv.extension('bokeh')

In [ ]:
dataArray = grid.fldE[0,:,:,:]

In [ ]:
dataArray.shape

In [ ]:
ds = hv.Dataset((np.arange(dataArray.shape[2]), np.arange(dataArray.shape[1]), np.arange(dataArray.shape[0]), dataArray ), ['x', 'y', 'z'], "z-component")

In [ ]:
hv.opts.defaults(
hv.opts.Image(cmap='viridis', fig_size=200.0, normalize=True, colorbar=True),
hv.opts.Labels(fontsize='8pt'),
    hv.opts.Path(color='white'),
    hv.opts.Spread(linewidth=600)
)

In [ ]:
ds.to(hv.Image, ['x','y'])

## Point charge field init

In [ ]:
from fdtd3d import UnifiedYeeGrid

In [ ]:
def pointCharge(x,y,z):
    factor = 1.0/(4*np.pi*eps0)
    return factor * np.array([x,y,z])/np.linalg.norm(np.array([x,y,z]))**3

In [ ]:
grid2 = UnifiedYeeGrid(Nx, Ny, Nz, pmlCells=(0,0,0), cellDim=(1./Cx, 1./Cy, 1./Cz) )

In [ ]:
grid2.initE(pointCharge)

In [ ]:
dataArray2 = np.linalg.norm(grid2.fldE, axis=0)

Define a symmetric logarithmic color scale. The scale is logarithmic above `linthrsh` and below `-linthrsh` and linear in between (since log-scale does not extend to 0). This linearization threshold is set to be the value of the 0.4-quantile of the magnitudes of the field data.

In [ ]:
from matplotlib.colors import LogNorm, SymLogNorm

In [ ]:
vmin = max(1e-40, abs(dataArray2).min())
vmax = max(1e-10, abs(dataArray2).max())

linthrsh = np.quantile(abs(dataArray2), 0.4)
forceNorm = SymLogNorm(vmax=vmax, vmin=-vmax, linthresh=linthrsh );

In [ ]:
#dataArray2 = np.linalg.norm(grid2.fldE, axis=0)
dataArray2 = grid2.fldE[2,:,:,:]

In [ ]:
ds2 = hv.Dataset((np.arange(grid2.originCellCoordinates[2], grid2.originCellCoordinates[2]+grid2.dz*grid2.Nz_virt, grid2.dz),
                  np.arange(grid2.originCellCoordinates[1], grid2.originCellCoordinates[1]+grid2.dy*grid2.Ny_virt, grid2.dy),
                  np.arange(grid2.originCellCoordinates[0], grid2.originCellCoordinates[0]+grid2.dx*grid2.Nx_virt, grid2.dx), (dataArray2) ), ['z', 'y', 'x'], "$ \\vec{E}_y$ [V/m]")

In [ ]:
hv.opts.defaults(
hv.opts.Image(cmap='viridis', fig_size=400.0, normalize=True, colorbar=True, norm=forceNorm),
hv.opts.Labels(fontsize='8pt'),
    hv.opts.Path(color='white'),
    hv.opts.Spread(linewidth=600)
)

In [ ]:
ds2.to(hv.Image, ['x', 'z'])

## Dipole field

Here we position a dipole consisting of $+e,-e$ around $\vec{0}$ at positions $x=\lbrace -0.02,0.02\rbrace$ and lot the z field component 

In [ ]:
def dipole(x,y,z):
    factor = 1.0/(4*np.pi*eps0)
    r1 = np.array([-0.02, 0,0 ])
    r2 = np.array([0.02, 0, 0])
    r11 = np.array([x,y,z]) - r1
    r21 = np.array([x,y,z]) - r2
    return factor * r11/np.linalg.norm(r11)**3 - factor*r21/np.linalg.norm(r21)**3

In [ ]:
grid3 = UnifiedYeeGrid(Nx, Ny, Nz, pmlCells=(0,0,0), cellDim=(1./Cx, 1./Cy, 1./Cz) )

In [ ]:
grid3.initE(dipole)

In [ ]:
dataArray3 = np.linalg.norm(grid3.fldE, axis=0)

In [ ]:
dataArray3 = grid3.fldE[2,:,:,:]

In [ ]:
ds3 = hv.Dataset((np.arange(grid3.originCellCoordinates[2], grid3.originCellCoordinates[2]+grid3.dz*grid3.Nz_virt, grid3.dz),
                  np.arange(grid3.originCellCoordinates[1], grid3.originCellCoordinates[1]+grid3.dy*grid3.Ny_virt, grid3.dy),
                  np.arange(grid3.originCellCoordinates[0], grid3.originCellCoordinates[0]+grid3.dx*grid3.Nx_virt, grid3.dx), (dataArray3) ), ['z', 'y', 'x'], "$ \\vec{E}_z$ [V/m]")

In [ ]:
ds3.to(hv.Image, ['x', 'z'])